# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "Meta AI",
    "industry": "Artificial Intelligence",
    "key_decision_maker": "Mark Zuckerberg",
    "position": "CEO",
    "milestone": "over half a billion users"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Meta AI, a company in the Artificial Intelligence sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather all available information about Meta AI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "Meta AI company overview"} 


Search results: Title: What Is Meta AI? What You Need to Know About the Social ... - CNET
Link: https://www.cnet.com/tech/services-and-software/what-is-meta-ai-what-you-need-to-know-about-the-social-networks-ai-tool



> Entering new CrewAgentExecutor chain...
I need to craft a personalized outreach campaign that resonates with Meta AI's company culture and addresses their recent achievements and future goals.

Action: Search the internet
Action Input: {"search_query": "Recent milestones of Meta AI"}Final Answer:

Subject: Elevating Meta AI's AI Capabilities with CrewAI's Solutions

Dear Mr. Zuckerberg,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI, a leading provider of advanced data analytics tools and AI-driven insights. As the CEO of Meta AI, I understand that you are constantly seeking ways to enhance your AI technologies and provide innovative solutions to your over half a billion users.

I recently came across the exciting news that Meta AI has reached a significant milestone with 400 million monthly active users across its platforms. This achievement is a testament to the exceptional work being done at Meta AI to engage with the AI community and advance g

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating Meta AI's AI Capabilities with CrewAI's Solutions

Dear Mr. Zuckerberg,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI, a leading provider of advanced data analytics tools and AI-driven insights. As the CEO of Meta AI, I understand that you are constantly seeking ways to enhance your AI technologies and provide innovative solutions to your over half a billion users.

I recently came across the exciting news that Meta AI has reached a significant milestone with 400 million monthly active users across its platforms. This achievement is a testament to the exceptional work being done at Meta AI to engage with the AI community and advance groundbreaking AI technologies. 

At CrewAI, we specialize in offering cutting-edge tools that can complement Meta AI's existing strategies and goals. Our solutions are designed to enhance AI models, improve user interactions, and optimize AI-driven platforms for increased efficiency and effectiveness. We believe that by leveraging our tools, Meta AI can further enhance its AI capabilities and provide even more value to its users.

In light of Meta AI's recent milestone and strategic focus on enhancing AI offerings, I would love to discuss how CrewAI's solutions can support your goals and contribute to the continued success of Meta AI. I am confident that a partnership between our two companies can drive mutual growth and innovation in the AI space.

I look forward to the opportunity to connect and explore potential collaboration opportunities. Please feel free to reach out to me at your convenience to schedule a meeting or discuss further details.

Thank you for your time and consideration.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

P.S. Here is a link to learn more about CrewAI's solutions: [Insert link to your company's website]